In [1]:
# %load_ext autoreload
# %autoreload
# %matplotlib inline

from news_crawler import NewsCrawler
import glob
import json
from IPython.core.debugger import set_trace
from pathlib import Path
from pprint import pprint
import spacy
from tokenizer import MergedTokenizer
import time

import gensim
import gensim.corpora as corpora

import pyLDAvis
import pyLDAvis.gensim as gensimvis

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

from tqdm.auto import tqdm
tqdm.pandas()

C:\Users\infomax\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
nlp = spacy.load('en_core_web_lg')

In [4]:
fnames = glob.glob('newsdata/downloaded/*.json')[:50]

In [4]:
merged_tokenizer = MergedTokenizer(nlp, merge_chunks=False, merge_ner=True)

In [5]:
data = []
for fname in tqdm(fnames):
    js = json.loads(Path(fname).read_text())
    doc = merged_tokenizer.lemmatized(js['text'])
    data.append(doc)

In [6]:
id2word = corpora.Dictionary(data)

In [7]:
corpus = [id2word.doc2bow(text) for text in data]

In [8]:
s = time.time()
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                            id2word=id2word, 
                                            num_topics=5, 
                                            random_state=100, 
                                            update_every=None, 
                                            chunksize=1, 
                                            passes=5, 
                                            alpha='auto', 
                                            per_word_topics=True)
pprint(time.time()-s)

8.586445569992065


In [11]:
lda_model = gensim.models.HdpModel(corpus=corpus, id2word=id2word)

In [12]:
lda_model.show_topics()

[(0,
  '0.012*say + 0.004*new + 0.004*time + 0.004*people + 0.003*trump + 0.003*president + 0.002*state + 0.002*company + 0.002*getty + 0.002*world + 0.002*come + 0.002*image + 0.002*tell + 0.002*go + 0.002*work + 0.002*good + 0.002*know + 0.002*report + 0.002*include + 0.002*high'),
 (1,
  '0.012*say + 0.004*trump + 0.004*time + 0.004*new + 0.003*people + 0.003*president + 0.003*good + 0.003*state + 0.003*go + 0.003*tell + 0.003*work + 0.002*come + 0.002*know + 0.002*want + 0.002*take + 0.002*house + 0.002*report + 0.002*think + 0.002*company + 0.002*world'),
 (2,
  '0.009*say + 0.003*time + 0.002*people + 0.002*new + 0.002*company + 0.002*work + 0.002*know + 0.002*go + 0.002*good + 0.002*start + 0.002*outstanding + 0.002*state + 0.001*game + 0.001*product + 0.001*come + 0.001*change + 0.001*shampoo + 0.001*tell + 0.001*want + 0.001*add'),
 (3,
  '0.007*say + 0.006*trump + 0.003*people + 0.003*president + 0.002*getty + 0.002*time + 0.002*new + 0.002*think + 0.002*chinese + 0.002*come 

In [17]:
pyLDAvis.enable_notebook()

In [ ]:
'''
여기를 실행시키려면 반드시 future 패키지를 0.18로 업그레이드한다
pip install future --upgrade
'''

vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

In [1]:
import glob
import json
import os
import re
import operator
import matplotlib.pyplot as plt
import warnings
import gensim
import numpy as np
# warnings.filterwarnings('ignore')

import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
from pprint import pprint
from smart_open import smart_open
from pathlib import Path

from tqdm.auto import tqdm
tqdm.pandas()

%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\infomax\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\infomax\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\infomax\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
def build_texts(date):
    fnames = glob.glob('newsdata/downloaded/*.json')
    for fname in tqdm(fnames[:100000]):
        js = json.loads(Path(fname).read_text())
        if js['published_at']==date:
            yield gensim.utils.simple_preprocess(js['text'], deacc=True, min_len=3)

In [5]:
train_texts = list(build_texts('2019-10-13'))

In [6]:
len(train_texts)

2308

In [7]:
bigram = gensim.models.Phrases(train_texts)

In [8]:
from gensim.utils import lemmatize
from nltk.corpus import stopwords

In [9]:
stops = set(stopwords.words('english'))

In [10]:
def process_texts(texts):
    texts = [[word for word in line if word not in stops] for line in texts]
    texts = [bigram[line] for line in texts]
    
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    
    #texts = [[word for word in lemmatizer.lemmatize(' '.join(line), pos='v').split()] for line in texts]
    texts = [[lemmatizer.lemmatize(word, pos='v') for word in line] for line in texts]
    return texts

In [11]:
texts = process_texts(train_texts)

In [12]:
texts[0]

['surprise',
 'one',
 'donald_trump',
 'factor',
 'heavily',
 'hate',
 'six',
 'part',
 'discovery',
 'channel',
 'documentary',
 'series',
 'executive',
 'produce',
 'steven_spielberg',
 'alex',
 'gibney',
 'investigate',
 'many',
 'form',
 'cause',
 'possible',
 'remedy',
 'intolerance',
 'ubiquitous',
 'presence',
 'fact',
 'even',
 'specific',
 'topic',
 'conversation',
 'corrosive',
 'shadow',
 'loom',
 'large',
 'proceed',
 'international',
 'criminal',
 'lawyer',
 'patricia',
 'viseur',
 'sellers',
 'describe',
 'khmer',
 'rouge',
 'madman',
 'pol',
 'pot',
 'motivations',
 'initiate',
 'genocide',
 'state',
 'pause',
 'indicate',
 'know',
 'imply',
 'go',
 'make',
 'cambodia',
 'great',
 'current',
 'president',
 'ugly',
 'divisive',
 'language',
 'policies',
 'white_nationalist',
 'movements',
 'mass',
 'shoot',
 'inspire',
 'natural',
 'fit',
 'directors',
 'geeta',
 'gandbhir',
 'sam',
 'pollard',
 'non_fiction',
 'work',
 'premier',
 'oct',
 'trump',
 'perfectly',
 'embody'

In [13]:
dictionary = Dictionary(texts)
dictionary.filter_extremes(no_below=10, no_above=0.5)
dictionary.compactify()

In [14]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [59]:
lsimodel = LsiModel(corpus=corpus, num_topics=10, id2word=dictionary)

In [36]:
lsimodel.show_topics(num_topics=5, formatted=False)

[(0,
  [('noon', -0.6613216968723826),
   ('lane', -0.46454260670204767),
   ('ave', -0.2841907711589004),
   ('way', -0.2583361514239686),
   ('lane_noon', -0.13663017844177874),
   ('hill', -0.1096466067462519),
   ('ave_noon', -0.09837372847808079),
   ('way_noon', -0.09837372847808079),
   ('park', -0.09349149630332552),
   ('county', -0.08767795208114522)]),
 (1,
  [('say', 0.49071339033150546),
   ('one', 0.16044114629089629),
   ('also', 0.1437593795008963),
   ('jesse', 0.1379700929949128),
   ('make', 0.13474691104042738),
   ('get', 0.11741075749882347),
   ('would', 0.11709819554865286),
   ('china', 0.11259388004910137),
   ('take', 0.10266960571547686),
   ('people', 0.10163539160551438)]),
 (2,
  [('jesse', -0.680802036078172),
   ('walt', -0.38078757950135045),
   ('say', 0.24994723380790815),
   ('gus', -0.11539017560646977),
   ('get', -0.10549742394335948),
   ('meth', -0.10385115804582276),
   ('breaking_bad', -0.09231214048517587),
   ('drug', -0.09191937442831924),

In [16]:
hdpmodel = HdpModel(corpus=corpus, id2word=dictionary, T=20)

In [17]:
hdpmodel.show_topics(formatted=False)

[(0,
  [('would', 0.004295570439678243),
   ('get', 0.004084392221812674),
   ('time', 0.0031267363718436194),
   ('take', 0.0031216896466816856),
   ('trump', 0.003086331857143635),
   ('people', 0.002989355167769212),
   ('go', 0.0027683690865242306),
   ('first', 0.0026692515921386325),
   ('come', 0.002633617384499611),
   ('back', 0.002588141019661147),
   ('two', 0.0025636002967385336),
   ('leave', 0.002489021551703571),
   ('could', 0.0024684194742144025),
   ('tell', 0.0024155139028360305),
   ('like', 0.0024112782699790207),
   ('new', 0.002320258854222873),
   ('work', 0.002264318715619079),
   ('sunday', 0.0021850215213367417),
   ('start', 0.0021762315781064055),
   ('call', 0.0021607537668540317)]),
 (1,
  [('syria', 0.0032018558555601487),
   ('turkey', 0.0026285795119861846),
   ('trump', 0.0025475580610988735),
   ('force', 0.0022063303996782903),
   ('would', 0.0021094460737535363),
   ('report', 0.0018491401382663712),
   ('northern_syria', 0.0018027722149259193),
  

In [35]:
ldamodel = LdaModel(corpus=corpus, 
                    num_topics=100, 
                    passes=20, 
                    id2word=dictionary, 
                    chunksize=2000, 
                    alpha='auto', 
                    eta='auto',
                    eval_every=None,
                    per_word_topics=True)

In [19]:
ldamodel.show_topics(formatted=False)

[(4,
  [('tokyo', 0.01942691),
   ('water', 0.014885103),
   ('sunday', 0.014591165),
   ('japan', 0.013930059),
   ('government', 0.013062565),
   ('cancel', 0.010487173),
   ('people', 0.010155819),
   ('nhk_said', 0.0097704725),
   ('typhoon_hagibis', 0.009549859),
   ('hit', 0.007848135)]),
 (12,
  [('afp_getty', 0.041581217),
   ('getty', 0.015870085),
   ('extinction_rebellion', 0.014875724),
   ('group', 0.012636563),
   ('protest', 0.012412991),
   ('london', 0.011933727),
   ('create', 0.00952538),
   ('part', 0.008996371),
   ('along', 0.008826856),
   ('climate_change', 0.008767581)]),
 (16,
  [('bitcoin', 0.0093893185),
   ('price', 0.008124783),
   ('would', 0.007704484),
   ('break', 0.0076032216),
   ('level', 0.007313293),
   ('move', 0.0073107346),
   ('show', 0.0073091313),
   ('weekly', 0.0068432693),
   ('come', 0.0066389814),
   ('end', 0.006572146)]),
 (9,
  [('cake', 0.024241915),
   ('japan', 0.019734893),
   ('getty_images', 0.01619749),
   ('side', 0.014265048

In [33]:
def ret_top_model():
    """
    Since LDAmodel is a probabilistic model, it comes up different topics each time we run it. To control the
    quality of the topic model we produce, we can see what the interpretability of the best topic is and keep
    evaluating the topic model until this threshold is crossed. 
    
    Returns:
    -------
    lm: Final evaluated topic model
    top_topics: ranked topics in decreasing order. List of tuples
    """
    top_topics = [(0, 0)]
    while top_topics[0][1] < 0.9:
        lm = LdaModel(corpus=corpus, id2word=dictionary, eval_every=None, num_topics=100, passes=20, alpha='auto', eta='auto', per_word_topics=True, chunksize=2000)
        coherence_values = {}
        for n, topic in lm.show_topics(num_topics=-1, formatted=False):
            topic = [word for word, _ in topic]
            cm = CoherenceModel(topics=[topic], texts=texts, dictionary=dictionary, window_size=10)
            coherence_values[n] = cm.get_coherence()
        top_topics = sorted(coherence_values.items(), key=operator.itemgetter(1), reverse=True)
        print(top_topics[0][1])
    return lm, top_topics

In [34]:
lm, top_topics = ret_top_model()

0.8101233016556432
0.7530478649782473
0.6938712938987388
0.7496064271145078
0.7341531118233621
0.7530478649782473
0.7068196405329843
0.8055868758230924
0.7530478649782473
0.7590263732732534
0.7530478649782473


KeyboardInterrupt: 

In [29]:
import pyLDAvis.gensim

In [30]:
pyLDAvis.enable_notebook()

In [31]:
# %%prun
vis = pyLDAvis.gensim.prepare(lm, corpus, dictionary)

C:\Users\infomax\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [32]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
15     0.185524 -0.016685       1        1  8.774457
47     0.065163 -0.087639       2        1  6.209461
10     0.158574  0.070551       3        1  4.739315
18    -0.006169 -0.152200       4        1  4.580611
24     0.144733 -0.063955       5        1  4.378047
8      0.150205  0.000483       6        1  3.829597
35     0.013426 -0.141749       7        1  3.796749
49     0.129458 -0.024204       8        1  3.261255
31     0.112219  0.068991       9        1  3.013697
20     0.065016 -0.109214      10        1  2.978101
19    -0.107373 -0.150564      11        1  2.940028
27    -0.008521 -0.067432      12        1  2.814304
0     -0.189608 -0.108962      13        1  2.597286
25     0.037845  0.170206      14        1  2.513941
32     0.154163  0.085665      15        1  2.512165
9      0.024697 -0.160888      16        1  2.494260
30     0.044433  0.194589      17        1  2.439309
37    -0.048776 -0.041790      18        1  2.293171
34    -0.107478 -0.010680      19        1  2.149442
46    -0.110911 -0.121439      20        1  2.035926
17    -0.028728 -0.056993      21        1  2.028993
48     0.143080  0.020734      22        1  1.981397
7      0.038854 -0.054489      23        1  1.884104
28     0.037147 -0.091760      24        1  1.734652
39    -0.058059 -0.042406      25        1  1.728211
3     -0.058379 -0.073225      26        1  1.623276
5      0.058911 -0.090966      27        1  1.455111
22    -0.008014 -0.023693      28        1  1.433119
26     0.026200 -0.070693      29        1  1.306927
6      0.039566  0.101276      30        1  1.302614
43     0.047545  0.061759      31        1  1.274117
2      0.119952  0.095622      32        1  1.229010
41     0.138751  0.022763      33        1  1.155179
33     0.005172  0.180043      34        1  1.057479
42    -0.175057  0.018385      35        1  0.945261
45    -0.082134 -0.012618      36        1  0.895840
44     0.069431 -0.026105      37        1  0.833330
36    -0.072492 -0.092546      38        1  0.824802
13     0.019527  0.013624      39        1  0.803627
4      0.072050  0.046589      40        1  0.604746
23     0.028323 -0.060975      41        1  0.547153
11    -0.035146  0.090209      42        1  0.544247
40    -0.171409  0.061691      43        1  0.481876
38    -0.119851  0.022252      44        1  0.418463
16    -0.113594  0.131967      45        1  0.391646
21    -0.099572  0.022427      46        1  0.390618
29    -0.143299  0.097806      47        1  0.272319
1     -0.092648  0.173727      48        1  0.220429
14    -0.139518  0.054917      49        1  0.181359
12    -0.153234  0.147592      50        1  0.098972, topic_info=     Category         Freq            Term        Total  loglift  logprob
242   Default  1905.000000           trump  1905.000000  30.0000  30.0000
643   Default   734.000000           fight   734.000000  29.0000  29.0000
973   Default   614.000000             hit   614.000000  28.0000  28.0000
96    Default  1216.000000            game  1216.000000  27.0000  27.0000
2952  Default   351.000000            ball   351.000000  26.0000  26.0000
...       ...          ...             ...          ...      ...      ...
7153  Topic50     0.023903  state_families     6.481126   1.3154 -10.0135
387   Topic50     0.023901           pitch    82.677666  -1.2307 -10.0136
954   Topic50     0.023899            data   244.472931  -2.3149 -10.0136
2197  Topic50     0.023889          league   135.367188  -1.7242 -10.0141
87    Topic50     0.023884           first  1649.638184  -4.2248 -10.0143

[3186 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1172      2  0.029619  abandon
1172      5  0.128349  abandon
1172      6  0.078984  abandon
1172     11  0.691112  abandon
1172     19  0.049365  abandon
...     ...       ...      ...
260       2  0.072403     z